In [1]:
from transformers import AutoTokenizer,AutoModelForCausalLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset
import torch
torch.cuda.empty_cache()
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import nltk
from sacrebleu import corpus_bleu
from nltk.translate.meteor_score import meteor_score
nltk.download('wordnet')  # Required for METEOR score
import random

# Load the pre-trained model and tokenizer
model_name = "Helsinki-NLP/opus-mt-ja-en"
original_model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")


data = load_dataset("NilanE/ParallelFiction-Ja_En-100k", split="train")

dataset = data.train_test_split(test_size=0.1, seed=42)
train_data = dataset['train']
test_data = dataset['test']

def preprocess_function(examples):
    # Extract Japanese source text and English target text
    inputs = examples['src']  # Japanese text
    targets = examples['trg']  # English text

    # Tokenize the source text
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    # Tokenize the target text as labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



# Function to preprocess test data
def preprocess_data(test_data, tokenizer):
    sources = []
    references = []
    for example in test_data:
        sources.append(example['src'])
        references.append([example['trg']])  # Wrap in a list for sacrebleu compatibility
    return sources, references

# Function to generate translations using the model
def generate_translations(model, tokenizer, sources):
    translations = []
    for source in sources:
        inputs = tokenizer(source, return_tensors="pt", truncation=True, padding=True).to(model.device)
        outputs = model.generate(
            **inputs,
            max_length=512,
            num_beams=4,
            length_penalty=1.2,
            no_repeat_ngram_size=3,
            early_stopping=True,   
            tempature = 0.6
        )
        translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
        translations.append(translation)
    return translations

def evaluate_model(model, tokenizer, test_data):
    # Preprocess data
    sources, references = preprocess_data(test_data, tokenizer)
    translations = generate_translations(model, tokenizer, sources)

    # BLEU score (using raw text)
    bleu_score = corpus_bleu(translations, references).score

    # Tokenize translations and references for METEOR
    tokenized_translations = [trans.split() for trans in translations]
    tokenized_references = [[ref.split() for ref in ref_list] for ref_list in references]

    # METEOR score
    meteor_scores = [
        max(meteor_score([ref], trans) for ref in ref_list)
        for ref_list, trans in zip(tokenized_references, tokenized_translations)
    ]
    avg_meteor_score = sum(meteor_scores) / len(meteor_scores)

    return bleu_score, avg_meteor_score



c:\Users\matth\anaconda3\envs\EpubTranslatorEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\matth\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
c:\Users\matth\anaconda3\envs\EpubTranslatorEnv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
tokenized_train_data = train_data.map(preprocess_function, batched=True)

tokenized_train_data.save_to_disk("./tokenized_data")

print(tokenized_train_data[0])


Saving the dataset (6/6 shards): 100%|██████████| 95443/95443 [00:01<00:00, 48119.72 examples/s]

{'src': '迷子のハイエルフ8\n転移ゲートに辿り着いてしまえばすぐにハイドラ王国である。今日はイベントのシーズンでもないのでゲートはそんなに混んでおらず、対して待ち時間も無く転移できた。\nハイドラ王国に付くとすぐに独特な海辺の匂いと喧騒が伝わってくる。\n「やっぱりハイドラ王国は何度きても賑やかですね」\n「巨大な港町と表現するのが正しいかもしれないね。実際海にほど近い市場は連日かなり賑やかなものさ。ハイドラ王国はいろいろ制度が革新的で、特に税金の比率が他に比べて低く個人で店を気軽に出しやすい条件が整っており、商売を始めやすいのが大きいね。ただ、その性質上個人から中小レベルの規模の店が多くなりがちで、大商会という規模のものは少ない傾向にあるね」\n「なるほど、確かにそんな雰囲気ですね」\nハイドラ王国は市民の力が強く、逆に貴族はほぼ名ばかりであまり権力を持っていないという状態らしい。いずれは貴族自体が無くなるのではないかとも言われており、やはり人界の中でも一番時代の先端を行く国のようだった。\n異世界出身の俺にとっては、比較的馴染みやすい気風かもしれない。\nそんなことを考えながら道を歩いていると、向こうから見覚えのあるおじいさんが歩いてきた。最初に会った時の変装しているラグナさんである。\n「フォルスさん、迎えが来たみたいですよ」\n「ああ、そのようだね。相変わらず変身魔法は下手だね......それなりに魔法に詳しいものが見ればすぐに違和感に気付くよ。まぁ、彼女はゴリッゴリの前衛タイプだから仕方ないと言えば仕方ないが......」\n「ノインさんも認識阻害とかは苦手って言ってましたね」\n「まぁ、魔法というものは得手不得手があるからね。私も賢者などという呼び名で呼ばれてはいるが、あらゆる魔法が使えたりというわけでもないさ。自然魔法や大地の魔法に特化していると言っていい。逆に火などの魔法は苦手だね」\nフォルスさんもラグナさんにはすぐ気付いたようで苦笑を浮かべていた。すると、ラグナさんの方も俺たちが気付いたことが分かったみたいで、軽く指で俺たちに合図を送ってきた。\nたぶんあの指の先の人気のない場所に行こうという合図だと思う。ラグナさんはとんでもない支持率を誇る国王であり、ハイドラ王国内では相当人気のある存在だ。\n実際俺と初めて会った時も変身魔法を使って

In [5]:
# Define training arguments with evaluation disabled
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",  # Disable evaluation
    save_strategy="steps",  # Save checkpoints periodically
    save_steps=500,  # Save a checkpoint every 500 steps
    save_total_limit=3,  # Keep only the last 3 checkpoints
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),  # Use FP16 if a GPU is available
    logging_dir="./logs",
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_train_data,  # Training data
    tokenizer=tokenizer,
)


c:\Users\matth\anaconda3\envs\EpubTranslatorEnv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\matth\AppData\Local\Temp\ipykernel_22360\3868523678.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# Fine-tune the model
trainer.train()

# Save the fine-tuned model and tokenizer
original_model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")


 42%|████▏     | 14896/35793 [3:34:55<211:27:27, 36.43s/it]

In [7]:
# Fine-tune the model
trainer.train(resume_from_checkpoint=True)

# Save the fine-tuned model and tokenizer
original_model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].
c:\Users\matth\anaconda3\envs\EpubTranslatorEnv\Lib\site-packages\transformers\trainer.py:3420: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recomm

{'loss': 2.1695, 'grad_norm': 2.804398775100708, 'learning_rate': 1.1622384265079764e-05, 'epoch': 1.26}


 43%|████▎     | 15500/35793 [16:15<4:43:53,  1.19it/s]

{'loss': 2.1699, 'grad_norm': 2.7245330810546875, 'learning_rate': 1.1343000027938424e-05, 'epoch': 1.3}


 45%|████▍     | 16000/35793 [23:22<4:25:46,  1.24it/s]

...

 99%|█████████▉| 35500/35793 [3:56:40<02:39,  1.83it/s]

{'loss': 1.9825, 'grad_norm': 2.5490450859069824, 'learning_rate': 1.7154192160478308e-07, 'epoch': 2.98}


100%|██████████| 35793/35793 [3:59:23<00:00,  2.49it/s]


{'train_runtime': 14363.9178, 'train_samples_per_second': 19.934, 'train_steps_per_second': 2.492, 'train_loss': 1.2199590257369632, 'epoch': 3.0}


('./fine_tuned_model\\tokenizer_config.json',
 './fine_tuned_model\\special_tokens_map.json',
 './fine_tuned_model\\vocab.json',
 './fine_tuned_model\\source.spm',
 './fine_tuned_model\\target.spm',
 './fine_tuned_model\\added_tokens.json')

In [4]:
test_data = test_data.shuffle(seed=42).select(range(100))

original_model = original_model.to("cuda" if torch.cuda.is_available() else "cpu")
bleu, meteor = evaluate_model(original_model, tokenizer, test_data)

print(f"BLEU Score: {bleu:.2f}")
print(f"METEOR Score: {meteor:.2f}")

finetuned_model = AutoModelForCausalLM.from_pretrained("fine_tuned_model")
finetuned_model = finetuned_model.to("cuda" if torch.cuda.is_available() else "cpu")
bleu, meteor = evaluate_model(finetuned_model, tokenizer, test_data)

print(f"BLEU Score: {bleu:.2f}")
print(f"METEOR Score: {meteor:.2f}")


OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 22.33 GiB is allocated by PyTorch, and 199.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)